In [65]:
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.utils import class_weight
from keras.models import Model, load_model
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from keras.layers import Input, Activation, Dense, concatenate, LSTM, GRU, Dropout

import xgboost as xgb
import datetime
import os
import joblib

# Get Data

In [2]:
def import_data():
    dfx = pd.read_csv('../data/x_train.csv').set_index('ID')
    dfy = pd.read_csv('../data/y_train.csv').set_index('ID')
    dfx_test = pd.read_csv('../data/x_test.csv').set_index('ID')
    return dfx, dfy, dfx_test

dfx, dfy, dfx_test = import_data()
dfx_test.head()

,neuron_id,timestamp_0,timestamp_1,timestamp_2,timestamp_3,timestamp_4,timestamp_5,timestamp_6,timestamp_7,timestamp_8,...,timestamp_40,timestamp_41,timestamp_42,timestamp_43,timestamp_44,timestamp_45,timestamp_46,timestamp_47,timestamp_48,timestamp_49
ID,,,,,,,,,,,,,,,,,,,,,
16635,10596,0.011162,0.221068,0.430589,0.502385,0.678627,1.022000,1.200516,1.496142,3.658803,...,9.694898,9.779988,15.699566,18.796651,19.081662,20.872090,22.135599,25.923326,34.213513,34.252010
16636,9836,0.461402,1.118278,1.708740,1.718111,1.723203,1.992070,2.203136,2.364779,2.875234,...,12.668427,12.679908,12.686186,13.058848,14.343126,14.737324,14.794357,16.582621,16.962624,17.078453
16637,10392,0.483416,0.574965,2.160823,2.271720,2.342907,2.885647,2.985719,3.059417,3.474729,...,18.373623,18.415545,18.553847,19.440229,20.054671,20.574787,20.729883,21.539690,22.917697,24.126154
16638,10045,0.008057,0.864662,1.610694,1.935644,1.947676,3.027078,3.237863,3.275289,3.300613,...,17.747281,18.363834,18.413200,18.528544,18.539123,18.995198,19.181297,19.436815,19.787004,19.929389
16639,8320,5.815709,7.666392,8.009023,8.027219,10.201623,10.249832,10.858486,10.865641,11.720478,...,28.048953,28.073936,28.080053,28.089389,29.077675,29.255499,29.438389,29.453067,29.516642,29.549979


In [3]:
def differencing(df):
    columns = df.columns
    new_df = np.concatenate((df.iloc[:,:2], df.iloc[:,2:].values - df.iloc[:,1:-1].values), axis=1)
    new_df = pd.DataFrame(new_df, columns=columns)
    new_df = new_df.drop(['timestamp_0'], axis=1)
    return new_df

dfx = differencing(dfx)
dfx_test = differencing(dfx_test)
dfx_test.head()

,neuron_id,timestamp_1,timestamp_2,timestamp_3,timestamp_4,timestamp_5,timestamp_6,timestamp_7,timestamp_8,timestamp_9,...,timestamp_40,timestamp_41,timestamp_42,timestamp_43,timestamp_44,timestamp_45,timestamp_46,timestamp_47,timestamp_48,timestamp_49
0,10596.0,0.209906,0.209521,0.071795,0.176242,0.343373,0.178515,0.295627,2.162661,0.297823,...,1.402280,0.085091,5.919577,3.097085,0.285011,1.790428,1.263509,3.787726,8.290187,0.038497
1,9836.0,0.656876,0.590462,0.009371,0.005092,0.268867,0.211066,0.161642,0.510456,0.154107,...,0.080638,0.011481,0.006279,0.372661,1.284278,0.394198,0.057033,1.788264,0.380003,0.115829
2,10392.0,0.091549,1.585858,0.110896,0.071187,0.542740,0.100072,0.073698,0.415311,0.030382,...,0.051218,0.041921,0.138303,0.886382,0.614442,0.520117,0.155096,0.809807,1.378007,1.208458
3,10045.0,0.856605,0.746032,0.324951,0.012032,1.079402,0.210785,0.037427,0.025324,0.005467,...,0.346384,0.616553,0.049366,0.115343,0.010579,0.456075,0.186098,0.255518,0.350189,0.142385
4,8320.0,1.850684,0.342631,0.018195,2.174404,0.048210,0.608654,0.007155,0.854837,0.375030,...,0.112346,0.024983,0.006117,0.009336,0.988286,0.177824,0.182890,0.014678,0.063575,0.033337


In [4]:
def sommes(df_diff, step="train"):
    """ Add 5 "sommes" (=quantiles) taken from the original spike trains to represent spike train length."""
    if step == "train":
        df = pd.read_csv('../data/x_train.csv').set_index('ID')
    else:
        df = pd.read_csv('../data/x_test.csv').set_index('ID')
        df_diff.index += 16635
        
    #somme_tokeep = [10, 20, 30, 40, 50]
    somme_tokeep = [25, 50]
    columns = ["somme_" + str(s) for s in somme_tokeep]
    somme = df.iloc[:,np.array(somme_tokeep)]
    somme.columns = columns
    new_df = pd.concat((df_diff, somme), axis=1)
    return new_df

dfx = sommes(dfx, "train")
dfx_test = sommes(dfx_test, "test")
dfx_test.head()

,neuron_id,timestamp_1,timestamp_2,timestamp_3,timestamp_4,timestamp_5,timestamp_6,timestamp_7,timestamp_8,timestamp_9,...,timestamp_42,timestamp_43,timestamp_44,timestamp_45,timestamp_46,timestamp_47,timestamp_48,timestamp_49,somme_25,somme_50
16635,10596.0,0.209906,0.209521,0.071795,0.176242,0.343373,0.178515,0.295627,2.162661,0.297823,...,5.919577,3.097085,0.285011,1.790428,1.263509,3.787726,8.290187,0.038497,5.706390,34.252010
16636,9836.0,0.656876,0.590462,0.009371,0.005092,0.268867,0.211066,0.161642,0.510456,0.154107,...,0.006279,0.372661,1.284278,0.394198,0.057033,1.788264,0.380003,0.115829,8.206950,17.078453
16637,10392.0,0.091549,1.585858,0.110896,0.071187,0.542740,0.100072,0.073698,0.415311,0.030382,...,0.138303,0.886382,0.614442,0.520117,0.155096,0.809807,1.378007,1.208458,7.351308,24.126154
16638,10045.0,0.856605,0.746032,0.324951,0.012032,1.079402,0.210785,0.037427,0.025324,0.005467,...,0.049366,0.115343,0.010579,0.456075,0.186098,0.255518,0.350189,0.142385,11.567272,19.929389
16639,8320.0,1.850684,0.342631,0.018195,2.174404,0.048210,0.608654,0.007155,0.854837,0.375030,...,0.006117,0.009336,0.988286,0.177824,0.182890,0.014678,0.063575,0.033337,22.351642,29.549979


In [5]:
def manual_features(df):
    df_diff = df.iloc[:,1:50]
    df["mean"] = df_diff.mean(axis=1)
    df["min"] = df_diff.min(axis=1)    
    df["max"] = df_diff.max(axis=1)
    quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    for q in quantiles:
        df["quant_" + str(q)] = df_diff.quantile(q, axis=1)
    return df

dfx = manual_features(dfx)
dfx_test = manual_features(dfx_test)

In [6]:
### VERY IMPORTANT STEP: Delete neuron_id, and timesteps columns  ###
"""
dfx = dfx.iloc[:,50:]
dfx_test = dfx_test.iloc[:,50:]
print(dfx.head())
"""

X_train, X_val, y_train, y_val = train_test_split(dfx, dfy, test_size=0.2, random_state=42)
y_train = np.squeeze(y_train) #add dimension to silence warnings from sklearn...
y_val = np.squeeze(y_val) #add dimension to silence warnings from sklearn...

In [7]:
fe_train = X_train.iloc[:,50:]
ts_train = X_train.iloc[:,:50]

fe_val = X_val.iloc[:,50:]
ts_val = X_val.iloc[:,:50]

# Train model

* **Random Forest:**

In [161]:
def train_rf(X_train, y_train):
    rf = RandomForestClassifier(random_state=42, class_weight='balanced')

    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 800, num = 10)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in range(1, 6)]
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]

    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}

    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 6, cv = 3, verbose=0, random_state=42, n_jobs = -1, scoring='roc_auc')
    rf_random.fit(X_train, y_train)

    best_random = rf_random.best_estimator_
    print("RF best CV ROC_AUC score: ", rf_random.best_score_)
    print("RF best params: ", rf_random.best_params_)
    return best_random

In [76]:
best_random = train_rf(X_train, y_train)
y_pred_val = best_random.predict(X_val)
print("ROC_AUC: ", metrics.roc_auc_score(y_val, y_pred_val))
print("CKS: ", metrics.cohen_kappa_score(y_val, y_pred_val))

RF best CV ROC_AUC score:  0.7814372088614453
RF best params:  {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 5, 'bootstrap': True}
ROC_AUC:  0.7285265209337685
CKS:  0.35930251751674835


* **XGBoost:**

In [ ]:
def train_xgb(X_train, y_train):
    # Class balancing trick
    scale_pos_weight = np.sum(y_train == 0)/ float(np.sum(y_train == 1))

    model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)

    params_XGB = {
            'min_child_weight': [1],
            'subsample': [0.6, 0.8, 1],
            'colsample_bytree': [0.6],
            'max_depth': [1],
            'n_estimators': [200],
            'learning_rate': [0.05, 0.1],
            'gamma': [0.5, 1],
            'scale_pos_weight': [scale_pos_weight]
            }

    search = RandomizedSearchCV(model, param_distributions=params_XGB, n_iter=1, scoring='roc_auc', n_jobs=4, 
                                verbose=0, random_state=1001 )

    search.fit(X_train, y_train)
    #search.fit(select_X_train, y_train)
    best_xgb = search.best_estimator_
    print("XGB best CV ROC_AUC score: ", search.best_score_)
    return best_xgb

In [ ]:
best_xgb = train_xgb(X_train, y_train)
y_pred_val = best_xgb.predict(X_val)
print("ROC_AUC: ", metrics.roc_auc_score(y_val, y_pred_val))
print("CKS: ", metrics.cohen_kappa_score(y_val, y_pred_val))

* **Predict:**

In [81]:
def predict(X_test, model):
    # Predict on custom X_test
    y_pred = model.predict(X_test)
    y_pred = np.reshape(y_pred, (y_pred.shape[0],))
    print (y_pred.shape)
    
    # Convert sigmoid output to 0s and 1s
    y_pred[y_pred >= 0.5] = 1
    y_pred[y_pred < 0.5] = 0
  
    # Format .csv in ENS style
    dfy_pred = pd.DataFrame(data=y_pred, columns=["TARGET"], dtype=int)
    dfy_pred.index.name = "ID"
    dfy_pred.index += 16635
    return dfy_pred

In [ ]:
dfy_pred = predict(dfx_test, best_xgb)
dfy_pred.iloc[:15]

In [ ]:
def saveExp(dfy_pred, model, params=None):
    """ Create directory in which to save predictions, experiment parameters and model object. """

    directory = "../experiments/{}".format(datetime.datetime.now().strftime("%m%d%H%M%S"))
    if not os.path.exists(directory):
        os.makedirs(directory)

    dfy_pred.to_csv(directory + '/y_pred.csv', sep=',')  
    joblib.dump(model, directory + '/model.h5')
    return directory

# Save model
saveExp(dfy_pred, best_xgb)

# Train groupby model

In [73]:
def groupby_neurons(df):
    # Get rid of the 50 spike train timesteps
    df_features = df.iloc[:, 50:]
    df_features["neuron_id"] = df["neuron_id"]
    
    # Group the samples by neuron_id
    grouped = df_features.groupby("neuron_id").mean()
    
    # Merge the features_df and the grouped_df on neuron_id column, and keep the original sample order
    final = df.reset_index().merge(grouped, on='neuron_id', sort=False).sort_values('index').set_index('index')
    return final

In [74]:
dfx, dfy, dfx_test = import_data()
dfx = differencing(dfx)
dfx_test = differencing(dfx_test)
dfx = sommes(dfx)
dfx_test = sommes(dfx_test, "test")
dfx = manual_features(dfx)
dfx_test = manual_features(dfx_test)
dfx_nid = groupby_neurons(dfx)
dfx_nid_test = groupby_neurons(dfx_test)
dfx_nid.head()

,neuron_id,timestamp_1,timestamp_2,timestamp_3,timestamp_4,timestamp_5,timestamp_6,timestamp_7,timestamp_8,timestamp_9,...,max_y,quant_0.1_y,quant_0.2_y,quant_0.3_y,quant_0.4_y,quant_0.5_y,quant_0.6_y,quant_0.7_y,quant_0.8_y,quant_0.9_y
index,,,,,,,,,,,,,,,,,,,,,
0,53.0,0.004258,0.005851,0.021194,0.015249,0.744817,0.220048,2.029589,0.006334,1.138758,...,5.829577,0.006480,0.010037,0.017278,0.038152,0.082100,0.186866,0.410137,0.995403,2.002561
1,7229.0,0.080382,0.027950,0.368929,0.162005,0.037755,0.053775,0.239245,0.035892,0.012870,...,3.449927,0.023353,0.043877,0.072804,0.115968,0.187514,0.288325,0.460189,0.771798,1.262314
2,7770.0,0.526874,0.162346,1.308847,1.228051,1.024309,0.174702,0.236532,0.020661,1.324287,...,2.604903,0.056812,0.098609,0.149527,0.213957,0.297770,0.409597,0.562768,0.784305,1.165241
3,7002.0,0.519697,0.336779,0.498451,0.755861,0.608494,0.352735,0.117038,0.877016,0.988270,...,2.161103,0.021653,0.050344,0.091367,0.146018,0.209738,0.301930,0.423963,0.607302,0.929014
4,7678.0,0.056557,0.030966,0.025940,0.172825,0.012688,0.035370,0.089025,0.015327,0.252605,...,2.959074,0.019778,0.047907,0.096047,0.163873,0.251328,0.372587,0.540685,0.785203,1.220172


In [75]:
def custom_split(dfx, dfy):
    """ Custom train_test_split function to keep the model from seeing the neuron_ids in X_val samples. """
    # Sample N random neuron_ids
    val_set = np.random.choice(pd.unique(dfx["neuron_id"]), 60, replace=False)
    val_idx = dfx['neuron_id'].isin(val_set)
    
    # Get X_train y_train, X_val y_val
    X_train = dfx[~val_idx].drop(["neuron_id"], axis=1)
    X_val = dfx[val_idx].drop(["neuron_id"], axis=1)
    y_train = dfy[~val_idx]
    y_val = dfy[val_idx]
    
    return X_train, X_val, y_train, y_val

X_train, X_val, y_train, y_val = custom_split(dfx_nid, dfy)
y_train = np.squeeze(y_train) #Delete useless dimension to silence warnings from sklearn...
y_val = np.squeeze(y_val) #Delete useless dimension to silence warnings from sklearn...

In [72]:
fe_train = X_train.iloc[:,50:]
ts_train = X_train.iloc[:,:50]

fe_val = X_val.iloc[:,50:]
ts_val = X_val.iloc[:,:50]

In [123]:
best_xgb_nid = train_xgb(X_train, y_train)
y_pred_val = best_xgb_nid.predict(X_val)
print("ROC_AUC: ", metrics.roc_auc_score(y_val, y_pred_val))
print("CKS: ", metrics.cohen_kappa_score(y_val, y_pred_val))

KeyboardInterrupt: 

In [166]:
best_random_nid = train_rf(X_train, y_train)
y_pred_val = best_random_nid.predict(X_val)
print("ROC_AUC: ", metrics.roc_auc_score(y_val, y_pred_val))
print("CKS: ", metrics.cohen_kappa_score(y_val, y_pred_val))

RF best CV ROC_AUC score:  0.8283823770348072
RF best params:  {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 5, 'bootstrap': True}
ROC_AUC:  0.7438107398132696
CKS:  0.2877495277717492


In [94]:
dfx_nid_test = dfx_nid_test.drop(["neuron_id"], axis=1)
dfy_pred = predict(dfx_nid_test, best_random_nid)
dfy_pred.iloc[:15]

(11969,)


,TARGET
ID,
16635,0
16636,0
16637,0
16638,0
16639,0
16640,0
16641,0
16642,0
16643,1


In [95]:
def saveExp(dfy_pred, model, params=None):
    """ Create directory in which to save predictions, experiment parameters and model object. """

    directory = "../experiments/{}".format(datetime.datetime.now().strftime("%m%d%H%M%S"))
    if not os.path.exists(directory):
        os.makedirs(directory)

    dfy_pred.to_csv(directory + '/y_pred.csv', sep=',')  
    joblib.dump(model, directory + '/model.h5')
    return directory

# Save model
saveExp(dfy_pred, best_random)

'../experiments/0630203950'

# LSTM

In [76]:
def balance_data(X, y, method="oversampling", **params):
    """ Return balanced training dataset obtained by undersampling class 2. """
    if method == "oversampling":
        sampler = RandomOverSampler(random_state=42)
    elif method == "undersampling":
        sampler = RandomUnderSampler(random_state=42)
    else:
        raise ValueError('Unrecognized sampling method: ', method)

    X, y = sampler.fit_sample(X, y)
    
    return X, y

X_train, y_train = balance_data(X_train, y_train)

In [77]:
fe_train = X_train[:,50:]
ts_train = X_train[:,:50]

fe_val = X_val.iloc[:,50:]
ts_val = X_val.iloc[:,:50]

ts_train = np.expand_dims(ts_train, axis=2)
ts_val = np.expand_dims(ts_val, axis=2)
print(ts_train.shape, ts_val.shape)

(23312, 50, 1) (2305, 50, 1)


In [78]:
timestep_nb = ts_train.shape[1]
spike_per_ts = 1
cell_nb = 256
dropout = 0.2

input_tensor = Input(shape=(timestep_nb, spike_per_ts))
X = LSTM(cell_nb, return_sequences=True, dropout=dropout)(input_tensor)
X = LSTM(cell_nb, return_sequences=False)(X)

additional_features = fe_train.shape[1]
fe_input = Input(shape=(additional_features,)) # A tensor containing the engineered features
latent = Dense(64, activation='relu')(fe_input)
latent = Dropout(rate=dropout)(latent)
latent = Dense(32, activation='relu')(latent)
latent = Dropout(rate=dropout)(latent)
input_tensor = [input_tensor, fe_input]
X = concatenate([X, latent])   
    
output_tensor = Dense(1, activation='sigmoid')(X)

model = Model(input_tensor, output_tensor)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 27)           0                                            
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 64)           1792        input_4[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 50, 1)        0                                            
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 64)           0           dense_4[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LS

In [79]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
model.compile(metrics=['accuracy'], loss='mse', optimizer='adam')
history = model.fit([ts_train, fe_train], y_train, class_weight=class_weights, epochs=1, validation_split=0.1, batch_size=32)

Train on 20980 samples, validate on 2332 samples
Epoch 1/1
20980/20980 [==============================] - 423s 20ms/step - loss: 0.2369 - acc: 0.6195 - val_loss: 0.2621 - val_acc: 0.5746


In [58]:
def predict(X_test, model):
    # Predict on custom X_test
    y_pred = model.predict(X_test)
    y_pred = np.reshape(y_pred, (y_pred.shape[0],))
    print (y_pred.shape)
    
    # Convert sigmoid output to 0s and 1s
    y_pred[y_pred >= 0.5] = 1
    y_pred[y_pred < 0.5] = 0
  
    # Format .csv in ENS style
    dfy_pred = pd.DataFrame(data=y_pred, columns=["TARGET"], dtype=int)
    dfy_pred.index.name = "ID"
    #dfy_pred.index += 16635
    return dfy_pred

In [80]:
y_pred_val = predict([ts_val, fe_val], model)
print("ROC_AUC: ", metrics.roc_auc_score(y_val, y_pred_val))
print("CKS: ", metrics.cohen_kappa_score(y_val, y_pred_val))

(2305,)
ROC_AUC:  0.6714195080687142
CKS:  0.2559211491659036


In [60]:
y_pred_val

,TARGET
ID,
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
